# Telcom Data

In [1]:
pip install psycopg2,openpyxl

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: 'psycopg2,openpyxl': Expected end or semicolon (after name and no valid version specifier)
    psycopg2,openpyxl
            ^


In [ ]:
import pandas as pd
import random
import psycopg2
from datetime import datetime, timedelta

In [ ]:
df = pd.read_excel('sample1.xlsx')

In [ ]:
df.head()

,1,6,6.1
0,2,7,14
1,3,8,24
2,4,9,36
3,5,10,50
4,6,11,66


In [ ]:
df.columns

In [56]:
len(df)

7043

In [57]:
df.dtypes

CustomerID            object
Count                  int64
Country               object
State                 object
City                  object
Zip Code               int64
Lat Long              object
Latitude             float64
Longitude            float64
Gender                object
Senior Citizen        object
Partner               object
Dependents            object
Tenure Months          int64
Phone Service         object
Multiple Lines        object
Internet Service      object
Online Security       object
Online Backup         object
Device Protection     object
Tech Support          object
Streaming TV          object
Streaming Movies      object
Contract              object
Paperless Billing     object
Payment Method        object
Monthly Charges      float64
Total Charges         object
Churn Label           object
Churn Value            int64
Churn Score            int64
CLTV                   int64
Churn Reason          object
dtype: object

In [58]:
try:
    conn = psycopg2.connect(
        database="telcom",
        user="postgres",
        password="postgres",
        host="localhost",
        port="5432"
    )
    print("Connected to PostgreSQL database successfully")

except psycopg2.Error as e:
    print("Error connecting to PostgreSQL database:", e)
    exit()

Connected to PostgreSQL database successfully


In [59]:
cur = conn.cursor()

In [60]:
query = "select * from customer_data"
try:
    cur.execute(query)
    customer_data = cur.fetchall()
except:
    customer_data = []
    conn.rollback()
if not customer_data:
    table_name = "customer_data"
    col_name = ["_".join(col.split()).lower() for col in df.columns]
    sql_query = f"INSERT INTO {table_name} ({', '.join(col_name)}) VALUES ({', '.join(['%s' for _ in range(len(df.columns))])})"
    for data in df.values:
        try:
            cur.execute(sql_query,data)
            conn.commit()
            print("Data inserted successfully.")
        except psycopg2.Error as e:
            print("Insertion error:",e)
            conn.rollback()
else:
    print(f"Already exist customer data")

Already exist customer data


In [61]:
try:
    cur.execute("SELECT customerid FROM customer_data")
    customers = cur.fetchall()
except psycopg2.Error as e:
    print("Error executing query:", e)
    conn.rollback()  # Rollback the transaction to clear the error state


In [62]:
customers[:5]

[('0002-ORFBO',),
 ('0003-MKNFE',),
 ('0004-TLHLJ',),
 ('0011-IGKFF',),
 ('0013-EXCHZ',)]

In [63]:
query = "SELECT * FROM customer_plans"
cur.execute(query)
plans = cur.fetchall()
if not plans:
    insert_query = """
    INSERT INTO customer_plans (customer_id, plan_id, start_date, end_date)
    VALUES (%s, %s, %s, %s)
"""
    for customer in customers:
        customer_id = customer[0]

        # Get a random plan from the telecom_plans table
        cur.execute("SELECT plan_id FROM telecom_plans ORDER BY random() LIMIT 1")
        plan = cur.fetchone()
        if plan is None:
            print("No plans available.")
            break
        plan_id = plan[0]

        # Generate a random start_date within the last 365 days
        random_days_offset = random.randint(0, 365)
        start_date = datetime.today() - timedelta(days=random_days_offset)

        # Generate a random end_date: at least 30 days after start_date and within the next year after start_date
        random_duration = random.randint(30, 365)
        end_date = start_date + timedelta(days=random_duration)

        try:
            cur.execute(insert_query, (customer_id, plan_id, start_date.date(), end_date.date()))
            conn.commit()
            print(f"Data inserted for customer {customer_id} with plan {plan_id}.")
        except psycopg2.Error as e:
            print("Insertion error for customer", customer_id, ":", e)
            conn.rollback()
else:
    print("Plans Already exists")

Plans Already exists


In [66]:
plan_query = "SELECT * FROM telecom_plans"
cur.execute(plan_query)
plans = cur.fetchall()

In [72]:
plans

[(1,
  'Basic Prepaid',
  'Prepaid',
  Decimal('19.99'),
  Decimal('2.00'),
  100,
  50,
  None,
  'Affordable plan for light users',
  datetime.datetime(2025, 3, 19, 21, 58, 57, 944264),
  datetime.datetime(2025, 3, 19, 21, 58, 57, 944264)),
 (2,
  'Unlimited Prepaid',
  'Prepaid',
  Decimal('49.99'),
  None,
  300,
  100,
  None,
  'Unlimited talk and text with unlimited data',
  datetime.datetime(2025, 3, 19, 21, 58, 57, 944264),
  datetime.datetime(2025, 3, 19, 21, 58, 57, 944264)),
 (3,
  'Family Prepaid',
  'Prepaid',
  Decimal('69.99'),
  Decimal('5.00'),
  200,
  150,
  None,
  'Plan for families with shared data and minutes',
  datetime.datetime(2025, 3, 19, 21, 58, 57, 944264),
  datetime.datetime(2025, 3, 19, 21, 58, 57, 944264)),
 (4,
  'Business Prepaid',
  'Prepaid',
  Decimal('89.99'),
  Decimal('8.00'),
  500,
  200,
  None,
  'Prepaid plan designed for small businesses',
  datetime.datetime(2025, 3, 19, 21, 58, 57, 944264),
  datetime.datetime(2025, 3, 19, 21, 58, 57, 

In [75]:
columns = [
    'plan_id',
    'plan_name',
    'plan_type',
    'price',
    'extra_fee',
    'talk_minutes',
    'text_messages',
    'data_limit',
    'description',
    'created_at',
    'updated_at'
]

In [77]:
customer_plans = pd.DataFrame(plans,columns=columns)

In [82]:
customer_plans

,plan_id,plan_name,plan_type,price,extra_fee,talk_minutes,text_messages,data_limit,description,created_at,updated_at
0,1,Basic Prepaid,Prepaid,19.99,2.00,100,50,NaN,Affordable plan for light users,2025-03-19 21:58:57.944264,2025-03-19 21:58:57.944264
1,2,Unlimited Prepaid,Prepaid,49.99,None,300,100,NaN,Unlimited talk and text with unlimited data,2025-03-19 21:58:57.944264,2025-03-19 21:58:57.944264
2,3,Family Prepaid,Prepaid,69.99,5.00,200,150,NaN,Plan for families with shared data and minutes,2025-03-19 21:58:57.944264,2025-03-19 21:58:57.944264
3,4,Business Prepaid,Prepaid,89.99,8.00,500,200,NaN,Prepaid plan designed for small businesses,2025-03-19 21:58:57.944264,2025-03-19 21:58:57.944264
4,5,Basic Postpaid,Postpaid,29.99,3.00,150,75,12.0,Entry-level postpaid plan for individual users,2025-03-19 21:58:57.944264,2025-03-19 21:58:57.944264
5,6,Unlimited Postpaid,Postpaid,59.99,None,400,150,12.0,Unlimited postpaid plan with all benefits,2025-03-19 21:58:57.944264,2025-03-19 21:58:57.944264
6,7,Family Postpaid,Postpaid,79.99,10.00,250,200,12.0,Family plan with extensive data sharing,2025-03-19 21:58:57.944264,2025-03-19 21:58:57.944264
7,8,Business Postpaid,Postpaid,99.99,12.00,600,250,12.0,Premium postpaid plan for businesses,2025-03-19 21:58:57.944264,2025-03-19 21:58:57.944264
8,9,Student Plan,Prepaid,15.99,1.50,80,40,NaN,Budget-friendly plan for students with moderat...,2025-03-19 21:58:57.944264,2025-03-19 21:58:57.944264
9,10,Senior Citizen Plan,Postpaid,24.99,2.50,120,60,12.0,Special postpaid plan with discounts for senio...,2025-03-19 21:58:57.944264,2025-03-19 21:58:57.944264


In [83]:
customer_plans['plan_name'].value_counts()

plan_name
Basic Prepaid            1
Unlimited Prepaid        1
Hybrid Plan              1
Super Saver Plan         1
Business Flex Plan       1
Urban Connect Plan       1
Rural Connect Plan       1
Corporate Plan           1
Unlimited Family Pack    1
Basic Family Pack        1
International Plus       1
Executive Plan           1
Value Plan               1
Budget Plan              1
Regional Plan            1
International Plan       1
SMS-Only Plan            1
Voice-Only Plan          1
Data-Only Plan           1
Premium Plan             1
Economy Plan             1
Senior Citizen Plan      1
Student Plan             1
Business Postpaid        1
Family Postpaid          1
Unlimited Postpaid       1
Basic Postpaid           1
Business Prepaid         1
Family Prepaid           1
Platinum Plan            1
Name: count, dtype: int64